<a href="https://colab.research.google.com/github/yckamra/MovieLens25m-Recommender/blob/main/Movie_Dictionary_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Installing project dependencies...")
!pip install numpy pandas scipy scikit-learn # Basic libraries we want
!pip install implicit # Installs implicit library for matrix factorization
#!pip install torch
print("Dependencies installed successfully.")

In [ ]:
from google.colab import auth
import zipfile
import os
import pandas as pd
import numpy as np
import implicit

In [ ]:
auth.authenticate_user()

!gcloud config set project movielens-recommender-451017

!gsutil cp gs://movielens-data/movielens_data.zip /content/

TMDb dataset: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv

MovieLens dataset: https://www.kaggle.com/datasets/garymk/movielens-25m-dataset

In [ ]:
!ls -lh /content/

In [ ]:
zip_path = "/content/movielens_data.zip"  # Change to your actual zip file name
extract_to = "/content/data/"  # Where to extract files

# Create directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete! Files are in:", extract_to)
!ls -lh /content/data/ml-25m

In [ ]:
genome_scores_CSV = "/content/data/ml-25m/genome-scores.csv"
genome_tags_CSV = "/content/data/ml-25m/genome-tags.csv"
links_CSV = "/content/data/ml-25m/links.csv"
movies_CSV = "/content/data/ml-25m/movies.csv"
ratings_CSV = "/content/data/ml-25m/ratings.csv"
tags_CSV = "/content/data/ml-25m/tags.csv"

In [ ]:
class Movie:
  # Items in self:
  #     ID
  #     name
  #     genre_list
  #     number_of_ratings -- total amount this movies has been rated
  #     sum_of_ratings -- summation of all the ratings from all users for this movie

  def __init__(self):
    self.ID = 0
    self.title = ""
    self.genre_list = []
    self.tag_list = []
    self.number_of_ratings = 0
    self.sum_of_ratings = 0

  def get_tag_list(self):
    return self.tag_list

  def add_tag_to_tag_list(self, tag):
    self.tag_list.append(tag)

  def get_ID(self):
    return self.ID

  def set_ID(self, ID):
    self.ID = ID

  def set_title(self, title):
    self.title = title

  def get_title(self):
    return self.title

  def set_genre_list(self, genre_list):
    self.genre_list = genre_list

  def add_genre_to_genre_list(self, genre):
    self.genre_list.append(genre)

  def get_genre_list(self):
    return self.genre_list

  def get_number_of_ratings(self):
    return self.number_of_ratings

  def set_number_of_ratings(self, number):
    self.number_of_ratings = number

  def add_to_number_of_ratings(self):
    self.number_of_ratings += 1

  def get_sum_of_ratings(self):
    return self.sum_of_ratings

  def set_sum_of_ratings(self, number):
    self.sum_of_ratings = number

  def add_rating_to_sum_of_ratings(self, rating):
    self.sum_of_ratings += rating

  def calculate_average_rating(self):
    return self.sum_of_ratings / self.number_of_ratings

In [ ]:
# Load ratings data
ratings_path = ratings_CSV
ratings_df = pd.read_csv(ratings_path)

print(ratings_df)

# Load movies data
movies_path = movies_CSV
movies_df = pd.read_csv(movies_path)

print(movies_df)

# Load tags data
tags_path = tags_CSV
tags_df = pd.read_csv(tags_path)

print(tags_df)

In [ ]:
def parse_genres(genre_string):
  genre_list = genre_string.split('|')
  return genre_list

def create_all_movies_dictionary(movies_df, ratings_df):
  all_movies_dictionary = {}
  for index, row in movies_df.iterrows():
    if row["movieId"] not in all_movies_dictionary:
      all_movies_dictionary[row["movieId"]] = Movie()
      all_movies_dictionary[row["movieId"]].set_title(row["title"])
      all_movies_dictionary[row["movieId"]].set_ID(row["movieId"])
      all_movies_dictionary[row["movieId"]].set_genre_list(parse_genres(row["genres"]))
  for index, row in tags_df.iterrows():
    if row["movieId"] not in all_movies_dictionary:
      all_movies_dictionary[row["movieId"]] = Movie()
    all_movies_dictionary[row["movieId"]].add_tag_to_tag_list(row["tag"])
  for index, row in ratings_df.iterrows():
    if row["movieId"] not in all_movies_dictionary:
      all_movies_dictionary[row["movieId"]] = Movie()
    all_movies_dictionary[row["movieId"]].add_to_number_of_ratings()
    all_movies_dictionary[row["movieId"]].add_rating_to_sum_of_ratings(row["rating"])

  return all_movies_dictionary

allMovies = create_all_movies_dictionary(movies_df, ratings_df)

In [ ]:
the_ID = 2
print(allMovies[the_ID].get_ID())
print(allMovies[the_ID].get_title())
print(allMovies[the_ID].get_genre_list())
print(allMovies[the_ID].calculate_average_rating())
print(allMovies[the_ID].get_number_of_ratings())
print(allMovies[the_ID].get_tag_list())